# Data

In [1]:
import sys
sys.path.append('..')

from src.data import make_paper_dataset, build_authors_df, scrape_google_scholar, enrich_author_df_with_scholarly

DATA_PATH = '../data'

In [2]:
# papers_df = make_paper_dataset(DATA_PATH)

In [3]:
# papers_df.groupby('conf')['doi'].apply(lambda g: g.isna().mean())

conf
FOCS    0.000000
ICML    0.331555
NIPS    1.000000
STOC    0.000000
Name: doi, dtype: float64

In [4]:
# authors_df = build_authors_df(papers_df)

/Users/shlomi/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


In [ ]:
# authors_with_co_df = authors_df[~authors_df['only_sing']]
# scholarly_results = scrape_google_scholar(authors_with_co_df, with_tor_proxy=True)
# authors_df = enrich_author_df_with_scholarly(scholarly_results, authors_df)
# authors_df.to_json(DATA_PATH + '/authors_FOCS_ICML_NIPS_STOC_2000_2009.json')  # TODO: Refactor

## Semantic Scholar

In [ ]:
# import time

import pandas as pd
#import semanticscholar as sch
#from tqdm.notebook import tqdm

# SLEEP = 1
# TIMEOUT = 10

#papers_df = pd.read_json('../data/papers_FOCS_ICML_NIPS_STOC_2000_2009.json')
#tcs_papers_df = papers_df[papers_df['community'] == 'TCS'].set_index('doi')


#paper_results = {}
#for doi in tqdm(missing_doi):
#    paper = sch.paper(doi, timeout=TIMEOUT)
#    if not paper:
#        print(doi)
#    paper_results[doi] = paper
#    time.sleep(SLEEP)
    
#sch_tcs_papers_df = pd.DataFrame.from_dict(paper_results, orient='index')   

#papers_df = pd.merge(papers_df,
#                     sch_tcs_papers_df.add_suffix('_sch'),
#                     how='left', left_on='doi', right_index=True)
#papers_df.to_json('../data/papers_FOCS_ICML_NIPS_STOC_2000_2009_with_sch.json')

In [10]:
import string
import numpy as np
import pandas as pd

papers_df = pd.read_json('../data/papers_FOCS_ICML_NIPS_STOC_2000_2009_with_sch.json')

In [12]:
papers_df.groupby('conf')['paperId_sch'].apply(lambda g: g.isna().mean())

conf
FOCS    0.001439
ICML    1.000000
NIPS    1.000000
STOC    0.000000
Name: paperId_sch, dtype: float64

In [13]:
len(papers_df[(~papers_df['authors_sch'].isna())])

1516

In [14]:
papers_df[(papers_df['conf'] == 'FOCS') & papers_df['paperId_sch'].isna()]

,authors,title,venue,pages,year,key,doi,ee,url,publisher,...,is_publisher_licensed_sch,paperId_sch,references_sch,title_sch,topics_sch,url_sch,venue_sch,year_sch,author_letter_int,avg_author_letter_int
610,[Mihai Patrascu],Succincter.,FOCS,305-313,2008,conf/focs/Patrascu08,10.1109/FOCS.2008.83,https://doi.org/10.1109/FOCS.2008.83,https://dblp.org/rec/conf/focs/Patrascu08,NaN,...,NaN,None,None,None,None,None,None,NaN,[15],15.0


In [15]:
all_authors = set.union(*papers_df[~papers_df['paperId_sch'].isna()]['authors_sch'].apply(lambda authors: {author['authorId']
                                                                                  for author in authors}))

In [ ]:
#author_results = {}

In [ ]:
# for author_id in tqdm(all_authors - set(author_results.keys())):
#    author = sch.author(author_id, timeout=TIMEOUT)
#    if not author:
#        print(author_id)
#    else:
#        author_results[author_id] = author
#    time.sleep(SLEEP)

In [ ]:
#with open('../data/authors_FOCS_ICML_NIPS_STOC_2000_2009_with_sch.pckl', 'wb') as f:
#    pickle.dump(author_results, f)

In [16]:
import pickle

authors_df = pd.read_json('../data/authors_FOCS_ICML_NIPS_STOC_2000_2009.json')
with open('../data/authors_FOCS_ICML_NIPS_STOC_2000_2009_with_sch.pckl', 'rb') as f:
    author_results = pickle.load(f)

In [17]:
def unify_sch_author_by_dblp(author_group):
    return pd.Series({'aliases': list(set.union(*map(set, author_group['aliases']))),
                      'authorId': list(author_group['authorId']),
                      'influentialCitationCount': sum(a for a in author_group['influentialCitationCount']),
                      'papers': sum(author_group['papers'], []),
                      'name': list(set(author_group['name'])),
                      'url': list(author_group['authorId'])})


authors_sch2dblp = {}
for _, paper in papers_df[~papers_df['authors_sch'].isna()].iterrows():
    authors_sch2dblp.update(dict(zip((author['authorId'] for author in paper['authors_sch']),
                                     paper['authors'])))
sch_tcs_authors_df = pd.DataFrame.from_dict(author_results, orient='index')
sch_tcs_authors_df['dblp_name'] = pd.Series(authors_sch2dblp)
assert (sch_tcs_authors_df.index == sch_tcs_authors_df['authorId']).all()
sch_tcs_authors_df = sch_tcs_authors_df.set_index('dblp_name')


sch_tcs_authors_df = pd.DataFrame.from_dict(author_results, orient='index')
sch_tcs_authors_df['dblp_name'] = pd.Series(authors_sch2dblp)

sch_tcs_authors_df = sch_tcs_authors_df.groupby('dblp_name').apply(unify_sch_author_by_dblp)
sch_tcs_authors_df['n_pub'] = sch_tcs_authors_df['papers'].apply(len)
sch_tcs_authors_df['scientific_birth_year'] = sch_tcs_authors_df['papers'].apply(lambda r: min(p['year'] for p in r if p['year']))
sch_tcs_authors_df['scientific_age'] = 2021 - sch_tcs_authors_df['scientific_birth_year']

sch_tcs_authors_df['annual_productiviy'] = sch_tcs_authors_df['n_pub'] / sch_tcs_authors_df['scientific_birth_year']

authors_df = pd.merge(authors_df,
             sch_tcs_authors_df.add_suffix('_sch'),
             how='left', left_index=True, right_index=True)

In [18]:
len(authors_df)

5422

In [19]:
from nameparser import HumanName
from ethnicolr import census_ln, pred_census_ln, pred_wiki_name, pred_wiki_ln

Using TensorFlow backend.


In [20]:
from nameparser import HumanName

authors_df = pd.merge(authors_df, authors_df['real_name'].apply(lambda r: HumanName(r)).apply(lambda name: pd.Series({'first_name': name.first,
                                                                           'middle_name': name.middle,
                                                                           'last_name': name.last})),
             how='left', left_index=True, right_index=True)

In [ ]:
from ethnicolr import pred_wiki_name
authors_df = pred_wiki_name(authors_df, 'last_name', 'first_name')
authors_df['race_perhaps'] = authors_df['race'].str.split(',').str[0]

In [43]:
# authors_df.to_json('../data/authors_FOCS_ICML_NIPS_STOC_2000_2009_with_sch.json')

In [37]:
# import gender_guesser.detector as gender

# gender_detector = gender.Detector()
# authors_df['last_name'].apply(gender_detector.get_gender)
# authors_df['perhaps_gender']